<html xmlns="http://www.w3.org/1999/xhtml"><head><title>Spark - evaluating history data</title><link rel="stylesheet" href="epub.css" type="text/css"/><meta name="generator" content="DocBook XSL Stylesheets V1.75.2"/></head><body id="page"><div class="section" title="Spark - evaluating history data"><div class="titlepage"><div><div><h1 class="title"><a id="ch10lvl1sec86"/>Spark - evaluating history data</h1></div></div></div><p>In this example, we combine the previous sections to look at some historical data and determine some useful attributes.</p><p>The historical data we are using is the guest list for <span class="emphasis"><em>The Jon Stewart Show</em></span>. A typical record from the data looks like this:</p><pre class="programlisting">
<span class="strong"><strong>1999,actor,1/11/99,Acting,Michael J. Fox</strong></span>
</pre><p>It contains the year, occupation of the guest, date of appearance, logical grouping of the occupation, and the name of the guest.</p><p>For our analysis, we will be looking at number of appearances per year, the most appearing occupation, and the most appearing personality.</p><p>We will be using this script:</p><pre class="programlisting">
<span class="strong"><strong>import pyspark</strong></span>
<span class="strong"><strong>import csv</strong></span>
<span class="strong"><strong>import operator</strong></span>
<span class="strong"><strong>import itertools</strong></span>
<span class="strong"><strong>import collections</strong></span>
<span class="strong"><strong>if not 'sc' in globals():</strong></span>
<span class="strong"><strong>    sc = pyspark.SparkContext()</strong></span>
<span class="strong"><strong>years = {}</strong></span>
<span class="strong"><strong>occupations = {}</strong></span>
<span class="strong"><strong>guests = {}</strong></span>
<span class="strong"><strong>#The file header contains these column descriptors</strong></span>
<span class="strong"><strong>#YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List</strong></span>
<span class="strong"><strong>with open('daily_show_guests.csv', 'rb') as csvfile:    </strong></span>
<span class="strong"><strong>    reader = csv.DictReader(csvfile)</strong></span>
<span class="strong"><strong>    for row in reader:</strong></span>
<span class="strong"><strong>        year = row['YEAR']</strong></span>
<span class="strong"><strong>        if years.has_key(year):</strong></span>
<span class="strong"><strong>            years[year] = years[year] + 1</strong></span>
<span class="strong"><strong>        else:</strong></span>
<span class="strong"><strong>            years[year] = 1</strong></span>
<span class="strong"><strong>        occupation = row['GoogleKnowlege_Occupation']</strong></span>
<span class="strong"><strong>        if occupations.has_key(occupation):</strong></span>
<span class="strong"><strong>            occupations[occupation] = occupations[occupation] + 1</strong></span>
<span class="strong"><strong>        else:</strong></span>
<span class="strong"><strong>            occupations[occupation] = 1</strong></span>
<span class="strong"><strong>        guest = row['Raw_Guest_List']</strong></span>
<span class="strong"><strong>        if guests.has_key(guest):</strong></span>
<span class="strong"><strong>            guests[guest] = guests[guest] + 1</strong></span>
<span class="strong"><strong>        else:</strong></span>
<span class="strong"><strong>            guests[guest] = 1</strong></span>
<span class="strong"><strong>syears = sorted(years.items(), key=operator.itemgetter(1), reverse=True)</strong></span>
<span class="strong"><strong>soccupations = sorted(occupations.items(), key=operator.itemgetter(1), reverse=True)</strong></span>
<span class="strong"><strong>sguests = sorted(guests.items(), key=operator.itemgetter(1), reverse=True)</strong></span>
<span class="strong"><strong>print syears[:5]</strong></span>
<span class="strong"><strong>print soccupations[:5]</strong></span>
<span class="strong"><strong>print sguests[:5]</strong></span>
</pre><p>The script has a number of features:</p><div class="itemizedlist"><ul class="itemizedlist"><li class="listitem" style="list-style-type: disc">We are using several packages.</li><li class="listitem" style="list-style-type: disc">It has the familiar context preamble.</li><li class="listitem" style="list-style-type: disc">We start dictionaries for the years, occupations, and guests. A dictionary contains a key and a value. For this use, the key will be the raw value from the CSV. The value will be the number of occurrences in the dataset.</li><li class="listitem" style="list-style-type: disc">We open the file and start reading line by line using a reader object.</li><li class="listitem" style="list-style-type: disc">On each line, we take the value of interest (<code class="literal">years</code>, <code class="literal">occupations</code>, <code class="literal">guests</code>):<div class="itemizedlist"><ul class="itemizedlist"><li class="listitem" style="list-style-type: disc">See whether the value is present in the appropriate dictionary</li><li class="listitem" style="list-style-type: disc">If it is there, increment the value (counter)</li><li class="listitem" style="list-style-type: disc">Otherwise, initialize an entry in the dictionary</li><li class="listitem" style="list-style-type: disc">We then sort each of the dictionaries in reverse order of the number of appearances of the item</li><li class="listitem" style="list-style-type: disc">Finally, we display the top five values for each dictionary</li></ul></div></li></ul></div><p>If we run this in a notebook, we have an output like this:</p><p>
</p><div class="mediaobject"><img src="graphics/image_10_008.jpg" alt="Spark - evaluating history data"/></div><p>
</p><p>We have the first part of our script down to handling the years and occupation accumulators. Here is the rest of the script:</p><p>
</p><div class="mediaobject"><img src="graphics/image_10_009.jpg" alt="Spark - evaluating history data"/></div><p>
</p><p>There may be a smarter way to do all of this, but I am not aware of it! The build-up of the accumulators is pretty standard, regardless of what language you are using. I think there is an opportunity to use a <code class="literal">map()</code> function here.&#xA0;We could add each collection to the Spark Context as lists and then apply <code class="literal">map.distinct.count</code>.</p><p>I really liked just trimming off the lists/arrays so easily instead of having to call some function. The number of guests per year is very consistent. Actors are prevalent-probably the people of most interest to the audience. The guest list was a little surprising. The guests are mostly actors, but I think all have strong political direction.</p></div></body></html>
